# Projeto 1 - Ciência dos Dados

Nome: Paulo Kim

Nome: Rodrigo Coelho

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'iPhone_X.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo iPhone_X.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train = train.iloc[:,:2]
train.head(5)

,Treinamento,Classificação
0,@iconredesign my prediction is the same but:\n...,2
1,still have mine but i never connected to my ph...,1
2,"iphone 11 na waste of money, just chill with x...",2
3,once the iphone 12 drops those using the iphon...,2
4,#ncat who fix’s phone ?? it’s a iphone x,1


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test = test.iloc[:,:2]
test.head(5)

,Teste,Classificação
0,@starrynight024 right?? that’s exactly what ha...,3
1,@gossipgirliee some of us haven't bought iphone x,2
2,@manyaww @mlgcollegefess hs without cond*m and...,1
3,very annoying that they didn't announce the ne...,2
4,@clarkvolo iphone x was released in november tho,2


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nós pegamos como produto o iPhone X. Para sua classificação, dividimos os tweets em:

* Propaganda/oferta (0)
* Muito irrelevante (1)
* Irrelevante (2)
* Relevante (3)
* Muito relevante (4)

TITULO LIMPEZA

In [5]:
# Cleanup
def cleanup(text):
    punctuation = '[!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    text_subbed_list = text_subbed.split()
    
    for word in text_subbed_list:
        if '@' in word or word[:5] == 'https':
            text_subbed_list.remove(word)
        else:
            pass
    return ' '.join(text_subbed_list)  

In [6]:
# Loop para realizar a limpeza
for i in range(len(train)):
    train.at[i,'Treinamento'] = cleanup(train.Treinamento[i]).lower() # (TOKKI) catei esse df.at do stack, ele basicamente altera o valor da respectiva row. Tava tentando outros
                                                                      # métodos, mas não tava funcionando, aí no final usei esse
    
train.head(10)

,Treinamento,Classificação
0,my prediction is the same but a13 bionic face ...,2
1,still have mine but i never connected to my ph...,1
2,iphone 11 na waste of money just chill with x ...,2
3,once the iphone 12 drops those using the iphon...,2
4,ncat who fix’s phone it’s a iphone x,1
5,[help] [iphone x] [ios 137] [checkra1n] when s...,1
6,rip iphone xyou were a great phone until i acc...,1
7,how do i reconnect to bluetooth on iphone x,1
8,my phone is xiaomi mi mix 2 also my phone is i...,1
9,bypass with iphones network blocked on activat...,0


In [7]:
# Calculando o total de palavras no conjunto dos tweets 
total_palavras = 0 
for phrase in train.Treinamento:
    total_palavras += len(phrase.split())

___
### Montagens das séries e tabelas relativas para cada classificação

Filtragem do dataframe para cada classficação, para assim montar as tabelas com as probabilidades de cada palavra dada uma classificação

(TOKKI) Aqui não gostei muito do jeito que eu fiz, queria ter feito algo mais "limpo", pode mudar se tiver uma ideia melhor 

In [8]:
# Transformando a lista de palavras da classficação Propaganda/Oferta (0) em uma série
lista_palavras_class_0 = []
palavras_class_0 = train.loc[train.Classificação == 0,:]

for phrase in palavras_class_0.Treinamento:
    lista_palavras_class_0 += phrase.split()
    
series_palavras_class_0 = pd.Series(lista_palavras_class_0)
series_palavras_class_0

0                      bypass
1                        with
2                     iphones
3                     network
4                     blocked
                ...          
7621                 tempered
7622                    glass
7623                     film
7624                    2pack
7625    https//tco/9pclfcslnu
Length: 7626, dtype: object

In [9]:
# Transformando a lista de palavras da classficação Muito Irrelevante (1) em uma série
lista_palavras_class_1 = []
palavras_class_1 = train.loc[train.Classificação == 1,:]

for phrase in palavras_class_1.Treinamento:
    lista_palavras_class_1 += phrase.split()
    
series_palavras_class_1 = pd.Series(lista_palavras_class_1)

In [10]:
# Transformando a lista de palavras da classficação Irrelevante (2) em uma série
lista_palavras_class_2 = []
palavras_class_2 = train.loc[train.Classificação == 2,:]

for phrase in palavras_class_2.Treinamento:
    lista_palavras_class_2 += phrase.split()
    
series_palavras_class_2 = pd.Series(lista_palavras_class_2)

In [11]:
# Transformando a lista de palavras da classficação Relevante (3) em uma série
lista_palavras_class_3 = []
palavras_class_3 = train.loc[train.Classificação == 3,:]

for phrase in palavras_class_3.Treinamento:
    lista_palavras_class_3 += phrase.split()
    
series_palavras_class_3 = pd.Series(lista_palavras_class_3)

In [12]:
# Transformando a lista de palavras da classficação Muito Relevante (4) em uma série
lista_palavras_class_4 = []
palavras_class_4 = train.loc[train.Classificação == 4,:]

for phrase in palavras_class_4.Treinamento:
    lista_palavras_class_4 += phrase.split()
    
series_palavras_class_4 = pd.Series(lista_palavras_class_4)

In [13]:
# Definindo as probabilidades das palavras dada a classificação
tabela_palavra_class_0 = series_palavras_class_0.value_counts()
tabela_palavra_class_1 = series_palavras_class_1.value_counts()
tabela_palavra_class_2 = series_palavras_class_2.value_counts()
tabela_palavra_class_3 = series_palavras_class_3.value_counts()
tabela_palavra_class_4 = series_palavras_class_4.value_counts()

In [14]:
print(tabela_palavra_class_1)

iphone            539
x                 395
the               293
i                 169
a                 142
                 ... 
aluminium           1
exchange            1
reduced             1
lg                  1
roadwayroadway      1
Length: 2332, dtype: int64


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [15]:
# Calculando a probabilddiade da palavra pertencer a respectiva classificação   
P_0 = len(lista_palavras_class_0) / total_palavras
P_1 = len(lista_palavras_class_1) / total_palavras
P_2 = len(lista_palavras_class_2) / total_palavras
P_3 = len(lista_palavras_class_3) / total_palavras
P_4 = len(lista_palavras_class_4) / total_palavras

In [16]:
# Smoothing
smoothing = {
    
    'V': 10**3,
    'alpha': 1
}

# Funções resposáveis em calcular as probabilidades com a aplicação do smoothing

def prob_frase_dado_class_0(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (tabela_palavra_class_0[palavra] + smoothing['alpha']) / (len(lista_palavras_class_0) + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (len(lista_palavras_class_0) + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

def prob_frase_dado_class_1(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (tabela_palavra_class_1[palavra] + smoothing['alpha']) / (len(lista_palavras_class_1) + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (len(lista_palavras_class_1) + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

def prob_frase_dado_class_2(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (tabela_palavra_class_2[palavra] + smoothing['alpha']) / (len(lista_palavras_class_2) + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (len(lista_palavras_class_2) + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

def prob_frase_dado_class_3(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (tabela_palavra_class_3[palavra] + smoothing['alpha']) / (len(lista_palavras_class_3) + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (len(lista_palavras_class_3) + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

def prob_frase_dado_class_4(frase):
    prob = 1
    for palavra in frase.split():
        try:
            prob *= (tabela_palavra_class_4[palavra] + smoothing['alpha']) / (len(lista_palavras_class_4) + smoothing['alpha']*smoothing['V'])
        except:
            prob *= smoothing['alpha'] / (len(lista_palavras_class_4) + smoothing['alpha']*smoothing['V'])
                                                                              
    return prob

In [17]:
def naive_bayes(tweet):
    p_0_dado_frase = prob_frase_dado_class_0(tweet) * P_0
    p_1_dado_frase = prob_frase_dado_class_1(tweet) * P_1
    p_2_dado_frase = prob_frase_dado_class_2(tweet) * P_2
    p_3_dado_frase = prob_frase_dado_class_3(tweet) * P_3
    p_4_dado_frase = prob_frase_dado_class_4(tweet) * P_4
    
    lista_de_probabilidades = [p_0_dado_frase,
                               p_1_dado_frase,
                               p_2_dado_frase,
                               p_3_dado_frase,
                               p_4_dado_frase]
    
    return lista_de_probabilidades.index(max(lista_de_probabilidades))

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [18]:
test.loc[:,'NBayes'] = 0
for i in range(len(test)):
    test.iloc[i,2] = naive_bayes(test.Teste[i])

In [32]:
test

,Teste,Classificação,NBayes
0,@starrynight024 right?? that’s exactly what ha...,3,4
1,@gossipgirliee some of us haven't bought iphone x,2,2
2,@manyaww @mlgcollegefess hs without cond*m and...,1,2
3,very annoying that they didn't announce the ne...,2,3
4,@clarkvolo iphone x was released in november tho,2,2
...,...,...,...
720,#mojosmartphone qi leather wireless charger ht...,0,0
721,shop #anythingyoucanimagine @takealot https://...,0,4
722,glowing apple logo backlit led case for iphone...,0,0
723,shit crazy had this iphone x forever https://t...,2,3


In [35]:
test.Teste[721]

'shop #anythingyoucanimagine @takealot https://t.co/2xlxbref9m https://t.co/eqkhoorbic'

In [36]:
cleanup(test.Teste[721])

'shop anythingyoucanimagine https//tco/2xlxbref9m'

In [37]:
naive_bayes(cleanup(test.Teste[721]))

0

___
## Concluindo:

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**